# Analysis Flow

This notebook demonstrates how an Automate Flow can be used to perform analysis. 
We use a flow that combines Globus Transfer and funcX to move data and then execute an analysis code on the data.

In [ ]:
from globus_automate_client import (create_flows_client, graphviz_format, state_colors_for_log,
                                    create_action_client, 
                                    create_flows_client)

import time
import json
import sys
import os

from funcx.sdk.client import FuncXClient

CLIENT_ID = "e6c75d97-532a-4c88-b031-8584a319fa3e"

## Create an analysis function

Use funcX to register a simple analysis function.

In [ ]:
# fxc = FuncXClient()
fxc = FuncXClient(use_offprocess_checker=False)

In [ ]:
import funcx
funcx.__version__

In [ ]:
def file_size(data):
    """Return the size of a file"""
    import os
    return os.path.getsize(data['pathname'])

func_uuid = fxc.register_function(file_size)
print(func_uuid)

In [ ]:
payload = {'pathname': '/etc/hostname'}
tutorial_endpoint = 'a694a32d-8707-4884-b205-3cfc87c4f667' # Public tutorial endpoint
res = fxc.run(payload, endpoint_id=tutorial_endpoint, function_id=func_uuid)
print(res)

In [ ]:
fxc.get_result(res)

## Create the flow

In [ ]:
flow_definition = {
  "Comment": "An analysis flow",
  "StartAt": "funcx",
  "States": {
    "funcx": {
      "Comment": "Run a funcX function",
      "Type": "Action",
      "ActionUrl": "https://api.funcx.org/automate",
      "ActionScope": "https://auth.globus.org/scopes/facd7ccc-c5f4-42aa-916b-a0e270e2c2a9/automate2",
      "Parameters": {
          "tasks": [{
            "endpoint.$": "$.input.fx_ep",
            "func.$": "$.input.fx_id",
            "payload.$": "$.input"
        }]
      },
      "ResultPath": "$.AnalyzeResult",
      "WaitTime": 600,
      "End": True
    }
  }
}


Register the flow

In [ ]:
flows_client = create_flows_client(CLIENT_ID)
flow = flows_client.deploy_flow(flow_definition, title="Stills process workflow")
flow_id = flow['id']
print(flow)
flow_scope = flow['globus_auth_scope']
print(f'Newly created flow with id:\n{flow_id}\nand scope:\n{flow_scope}')

In [ ]:
flow_input = {
    "input": {
        "fx_id": func_uuid,
        "fx_ep": tutorial_endpoint,
        "pathname": '/etc/hostname'
    }
}

Run the flow and wait for it to complete

Note: This will try to transfer a file called "test.txt" from your Globus Tutorial Endpoint 1.

In [ ]:
flow_action = flows_client.run_flow(flow_id, flow_scope, flow_input)
# print(flow_action)
flow_action_id = flow_action['action_id']
flow_status = flow_action['status']
print(f'Flow action started with id: {flow_action_id}')
while flow_status == 'ACTIVE':
    time.sleep(5)
    flow_action = flows_client.flow_action_status(flow_id, flow_scope, flow_action_id)
    flow_status = flow_action['status']
    print(f'Flow status: {flow_status}')

Check the result of the analysis function

In [ ]:
flow_action['details']